In [ ]:
# Dependencies
import requests
import json
import os
import pandas as pd
import hvplot.pandas
import pprint
import cartopy.crs as ccrs
# Load JSON
filepath = os.path.join("Resources", "USDM.json")
with open(filepath) as jsonfile:
    json_data = json.load(jsonfile)
#json_data
url = "https://www.fire.ca.gov/umbraco/api/IncidentApi/List?"
incident_acres_burned = []
incident_latitude = []
incident_longitude = []
incident_start_timestamp = []

start_year = 2021
end_year = 2022
counter = start_year

def json_find(year):
    add = f"year={year}&inactive=true"
    response = requests.get(url + add)
    data = response.json()
    return data

def parse_data(year):
    for response in range(len(year)):
        incident = year[response]
        AcresBurned = incident["AcresBurned"]
        latitude = incident["Latitude"]
        longitude = incident["Longitude"]
        timestamp = incident["Started"]
        if AcresBurned != None:
            if (longitude <= -110) & (longitude >= -130):
                if (latitude <= 43) & (latitude >= 32):
                    incident_acres_burned.append(AcresBurned)
                    incident_latitude.append(latitude)
                    incident_longitude.append(longitude)
                    incident_start_timestamp.append(timestamp)

while counter <= end_year:
    this_json = json_find(counter)
    parse_data(this_json)
    counter += 1
test = pd.DataFrame(data={'Acres Burned': incident_acres_burned, 'Latitude': incident_latitude, 
                          'Longitude': incident_longitude, 'Incident Start': incident_start_timestamp})
test.head()

In [ ]:
add = f"year=2022&inactive=true"
response = requests.get(url + add)
data = response.json()
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(data[0])

In [ ]:
mean = test.mean()
median = test.median()
print(f'Mean: {mean}')
print(f'Median: {median}')
test['Acres Burned'].describe().astype('int64')

In [ ]:
test['Latitude'].head()

In [ ]:
test['Longitude'].head()

In [ ]:
map_plot = test.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    hover_cols = "Acres Burned",
    size = "Acres Burned",
    frame_width = 3840,
    frame_height = 2160,
    scale = 0.05
)

In [ ]:
map_plot